In [ ]:
import numpy as np
from ECMWFAnalysis import *
import datetime

import pandas as pd

from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

import cv2, os

from scipy.spatial import KDTree

In [ ]:
def Weibull(x, λ, k):
    
    return 1 - np.exp(-(x/λ)**k)

def probability_animation(filenames, ts, labels, anim_name, fps, shape=(101, 151)):

    extend = [-10, 5, 45, 35]

    cbar_ticks = np.arange(0, 1.1, 0.1)
    
    clevels = np.linspace(0, 1, 100)

    dpi = 300
    
    cmap= plt.cm.jet 

    cbar_label="Hatching probability"

    #-- create figure and axes instances
    fig = plt.figure(figsize=(7, 4), dpi=dpi)
    ax  = fig.add_axes([0.1,0.1,0.8,0.9])

    projection = ccrs.PlateCarree()

    #-- create map
    ax = plt.axes(projection=projection) 

    #-- add map features
    ax.coastlines(resolution='10m') #110m, 50m, 10m
    ax.add_feature(cartopy.feature.LAND, edgecolor='black')
    ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black')
    ax.add_feature(cartopy.feature.OCEAN, zorder=100, edgecolor='black', facecolor='azure')
    ax.add_feature(cartopy.feature.STATES, edgecolor='black')  # provincias y CCAA

    GDD, lons, lats = np.loadtxt(filenames[0], unpack=True)
    lons = lons.reshape(shape)
    lats = lats.reshape(shape)
    
    prob = Weibull(GDD, 164.86, 4.34)
    
    cnplot = ax.pcolormesh(lons, lats, prob.reshape(lats.shape), cmap=cmap, vmin=0, vmax=1)
    
    cbar = plt.colorbar(cnplot, pad = 0.06, label=cbar_label)
    
    gl = ax.gridlines(crs=projection, linewidth=1, color='gray', alpha=0.5, zorder=200, linestyle='--')

    gl.xlabel_style = {'size': 8, 'color': 'gray'}
    gl.ylabel_style = {'size': 8, 'color': 'gray'}
    gl.bottom_labels = True
    gl.left_labels = True

    ax.set_extent(extend)
    
    GDD_acc = np.zeros(101*151)

    def animate(i, GDD_acc):
        
        print("Computing %s" % dates[i])
        
        GDD, lons, lats = np.loadtxt(filenames[i], unpack=True)
        
        lons = lons.reshape(shape)
        lats = lats.reshape(shape)
        
        GDD_acc += GDD
        
        prob = Weibull(GDD_acc, 164.86, 4.34)
                
        ax.pcolormesh(lons, lats, prob.reshape(lats.shape), cmap=cmap, vmin=0, vmax=1)
        
        plt.title(labels[i].strftime('%d/%m/%Y'))


    ani = FuncAnimation(fig, animate, frames=ts, interval=100, fargs=(GDD_acc,))

    ani.save('%s.mp4' % anim_name, writer=animation.FFMpegWriter(fps=fps))

    plt.close(fig)


In [ ]:
YEAR = 2018

start = datetime.datetime(YEAR-1, 12, 1)
end = datetime.datetime(YEAR, 2, 1)

filenames = []
dates = []
ts = []

current_date = start

i = 0

while current_date < end:
    
    year = current_date.strftime('%Y')

    folder = "GDD_data/%s" % year
    
    filename = folder + "/PS_GDD_" + current_date.strftime('%-d_%-m_%Y') + ".txt"
    
    filenames.append(filename)
    dates.append(current_date)
    ts.append(i)
    
    current_date += timedelta(1)
    
    i += 1
    
anim_name = "Egg_hatching_probability"
fps = 4

probability_animation(filenames, ts, dates, anim_name, fps)